In [1]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 31.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.8/294.8 kB 30.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 88.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 67.9 MB/s eta 0:00:00


In [2]:
import torch
import torch.nn as nn
from transformers import AutoTokenizer, AutoModel,AutoModelForSequenceClassification
import pandas as pd
import torch.nn.functional as F
import numpy as np

In [3]:
if torch.cuda.is_available():
    device = torch.device("cuda")
    print("GPU is available and being used")
else:
    device = torch.device("cpu")
    print("GPU is not available, using CPU instead")

GPU is available and being used


In [4]:
import os

os.environ['CUDA_VISIBLE_DEVICES']  = "0"

torch.cuda.device_count()

1

In [5]:
PATH = '/content/drive/MyDrive/Kaggle/CommonLit - Evaluate Student Summaries/commonlit-evaluate-student-summaries/'

In [6]:
prompts_train = pd.read_csv(PATH + "prompts_train.csv")
prompts_test = pd.read_csv(PATH + "prompts_test.csv")

summaries_train = pd.read_csv(PATH + "summaries_train.csv")
summaries_test = pd.read_csv(PATH + "summaries_test.csv")

sample_submission = pd.read_csv(PATH +"sample_submission.csv")

prompts_train

,prompt_id,prompt_question,prompt_title,prompt_text
0,39c16e,Summarize at least 3 elements of an ideal trag...,On Tragedy,Chapter 13 \r\nAs the sequel to what has alrea...
1,3b9047,"In complete sentences, summarize the structure...",Egyptian Social Structure,Egyptian society was structured like a pyramid...
2,814d6b,Summarize how the Third Wave developed over su...,The Third Wave,Background \r\nThe Third Wave experiment took ...
3,ebad26,Summarize the various ways the factory would u...,Excerpt from The Jungle,"With one member trimming beef in a cannery, an..."


In [7]:
# dataframe 병합, prompt id 기준으로 각 아이디에 맞는 prompts_train, test 내용을 왼쪽에 붙임
train = summaries_train.merge(prompts_train, how="left", on="prompt_id")
test = summaries_test.merge(prompts_test, how="left", on="prompt_id")

train

,student_id,prompt_id,text,content,wording,prompt_question,prompt_title,prompt_text
0,000e8c3c7ddb,814d6b,The third wave was an experimentto see how peo...,0.205683,0.380538,Summarize how the Third Wave developed over su...,The Third Wave,Background \r\nThe Third Wave experiment took ...
1,0020ae56ffbf,ebad26,They would rub it up with soda to make the sme...,-0.548304,0.506755,Summarize the various ways the factory would u...,Excerpt from The Jungle,"With one member trimming beef in a cannery, an..."
2,004e978e639e,3b9047,"In Egypt, there were many occupations and soci...",3.128928,4.231226,"In complete sentences, summarize the structure...",Egyptian Social Structure,Egyptian society was structured like a pyramid...
3,005ab0199905,3b9047,The highest class was Pharaohs these people we...,-0.210614,-0.471415,"In complete sentences, summarize the structure...",Egyptian Social Structure,Egyptian society was structured like a pyramid...
4,0070c9e7af47,814d6b,The Third Wave developed rapidly because the ...,3.272894,3.219757,Summarize how the Third Wave developed over su...,The Third Wave,Background \r\nThe Third Wave experiment took ...
...,...,...,...,...,...,...,...,...
7160,ff7c7e70df07,ebad26,They used all sorts of chemical concoctions to...,0.205683,0.380538,Summarize the various ways the factory would u...,Excerpt from The Jungle,"With one member trimming beef in a cannery, an..."
7161,ffc34d056498,3b9047,The lowest classes are slaves and farmers slav...,-0.308448,0.048171,"In complete sentences, summarize the structure...",Egyptian Social Structure,Egyptian society was structured like a pyramid...
7162,ffd1576d2e1b,3b9047,they sorta made people start workin...,-1.408180,-0.493603,"In complete sentences, summarize the structure...",Egyptian Social Structure,Egyptian society was structured like a pyramid...
7163,ffe4a98093b2,39c16e,An ideal tragety has three elements that make ...,-0.393310,0.627128,Summarize at least 3 elements of an ideal trag...,On Tragedy,Chapter 13 \r\nAs the sequel to what has alrea...


In [8]:
test

,student_id,prompt_id,text,prompt_question,prompt_title,prompt_text
0,000000ffffff,abc123,Example text 1,Summarize...,Example Title 1,Heading\nText...
1,111111eeeeee,def789,Example text 2,Summarize...,Example Title 2,Heading\nText...
2,222222cccccc,abc123,Example text 3,Summarize...,Example Title 1,Heading\nText...
3,333333dddddd,def789,Example text 4,Summarize...,Example Title 2,Heading\nText...


In [9]:
train['merged_text'] = 'text '+train['text'] + ' prompt_question ' + train['prompt_question'] + ' prompt_title '+train['prompt_title']+ ' prompt_text '+train['prompt_text']
train['merged_text'].head(2)

0    text The third wave was an experimentto see ho...
1    text They would rub it up with soda to make th...
Name: merged_text, dtype: object

In [10]:
test['merged_text'] = 'text '+ test['text'] + ' prompt_question ' + test['prompt_question'] + ' prompt_title '+ test['prompt_title']+ ' prompt_text '+ test['prompt_text']
test['merged_text'].head(2)

0    text Example text 1 prompt_question Summarize....
1    text Example text 2 prompt_question Summarize....
Name: merged_text, dtype: object

In [11]:
tokenizer = AutoTokenizer.from_pretrained('roberta-base')
model = AutoModel.from_pretrained('roberta-base')

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [12]:
class CFG:
    def __init__(self):
        self.model_name = model
        self.tokenizer_path = tokenizer
        self.batch_size = 16
        self.num_epochs = 40
        self.token_max = 512
        self.learning_rate = 1e-5
        self.model_save_path = PATH + 'model.pt'

cfg = CFG()

In [13]:
from itertools import chain
from torch.utils.data import TensorDataset, DataLoader, random_split

In [14]:
def get_dataset(df):
    encoding = tokenizer(list(train['merged_text']),
                   padding=True,
                   truncation=True,
                   max_length=cfg.token_max,
                   return_tensors="pt")
    content = torch.tensor(df['content'], dtype=torch.float32)
    wording = torch.tensor(df['wording'], dtype=torch.float32)
    labels = torch.dstack((content, wording)).squeeze()
    dataset = TensorDataset(encoding['input_ids'], encoding['attention_mask'], labels)
    return dataset

ds = get_dataset(train)

In [15]:
class Roberta(nn.Module):
    def __init__(self):
        super(Roberta, self).__init__()
        self.Roberta = model

        self.dropout = nn.Dropout(0.1)
        self.linear1 = nn.Linear(768, 128)
        self.linear2 = nn.Linear(128, 2)

    def forward(self, input_ids, attention_mask):
        outputs = self.Roberta(input_ids=input_ids, attention_mask=attention_mask) # roberta는 tokentypeid가 없음
        pooled_output = outputs.pooler_output
        pooled_output = self.dropout(pooled_output)
        output = self.linear1(pooled_output)
        output = nn.ReLU()(output)
        output = self.linear2(output)
        return output

In [16]:
train_ds, val_ds = random_split(ds, [0.8, 0.2])
print(len(train_ds))
print(len(val_ds))

5732
1433


In [17]:
train_dataloader = DataLoader(
            train_ds,
            batch_size = cfg.batch_size
)

val_dataloader = DataLoader(
            val_ds,
            batch_size = cfg.batch_size
)

In [20]:
model = Roberta().to("cuda")
optimizer = torch.optim.AdamW(model.parameters(), lr=cfg.learning_rate)
loss_fn = nn.MSELoss() # RMSE
min_val_loss = np.inf

In [21]:
model

Roberta(
  (Roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((

In [22]:
# Training loop
model.train()
patience = 0

for epoch in range(cfg.num_epochs):
    running_loss = 0.0
    for step, (input_ids, attention_mask, labels) in enumerate(train_dataloader):
        input_ids = input_ids.to("cuda")
        attention_mask = attention_mask.to("cuda")
        labels = labels.to("cuda")

        optimizer.zero_grad()

        outputs = model(input_ids, attention_mask)
        loss = loss_fn(outputs, labels)
        loss.backward()
        optimizer.step()
        if step % 100 == 0:
            print("Epoch {}, Step {}, Loss: {}".format(epoch+1, step, loss.item()))

        running_loss += loss.item()

    print(f"Epoch {epoch+1} Loss: {running_loss / len(train_dataloader)}")

    # Validation loop
    model.eval()
    with torch.no_grad():
        val_loss = 0.0
        for val_step, (input_ids, attention_mask, labels) in enumerate(val_dataloader):
            input_ids = input_ids.to("cuda")
            attention_mask = attention_mask.to("cuda")
            labels = labels.to("cuda")

            val_outputs = model(input_ids, attention_mask)
            val_loss += loss_fn(val_outputs, labels)
        avg_loss = val_loss / len(val_dataloader)
        print(f"Validation Loss: {avg_loss}")

        if avg_loss < min_val_loss:
            patience = 0
            min_val_loss = avg_loss
            torch.save(model.state_dict(), cfg.model_save_path)
            print(f'saving model with score: {avg_loss}')

    patience += 1
    if patience >= 10:
            print('Early Stopping trigerred on epoch: {}')
            break

    model.train()

Epoch 1, Step 0, Loss: 1.4231376647949219
Epoch 1, Step 100, Loss: 0.46499067544937134
Epoch 1, Step 200, Loss: 0.3279474377632141
Epoch 1, Step 300, Loss: 0.2837844491004944
Epoch 1 Loss: 0.5211119623154319
Validation Loss: 0.32447776198387146
saving model with score: 0.32447776198387146
Epoch 2, Step 0, Loss: 0.5091434121131897
Epoch 2, Step 100, Loss: 0.22547216713428497
Epoch 2, Step 200, Loss: 0.17048951983451843
Epoch 2, Step 300, Loss: 0.20392337441444397
Epoch 2 Loss: 0.26818164286507207
Validation Loss: 0.26995643973350525
saving model with score: 0.26995643973350525
Epoch 3, Step 0, Loss: 0.32525455951690674
Epoch 3, Step 100, Loss: 0.15859633684158325
Epoch 3, Step 200, Loss: 0.16140565276145935
Epoch 3, Step 300, Loss: 0.18043403327465057
Epoch 3 Loss: 0.2238588589853231
Validation Loss: 0.2640882730484009
saving model with score: 0.2640882730484009
Epoch 4, Step 0, Loss: 0.27176034450531006
Epoch 4, Step 100, Loss: 0.17806071043014526
Epoch 4, Step 200, Loss: 0.13046145439

In [25]:
from collections import OrderedDict

In [27]:
model = Roberta().cuda()
#model.load_state_dict(torch.load(cfg.model_save_path))

'''if isinstance(model, nn.DataParallel):
  model.load_state_dict(torch.load('model.pt'))
else:
  state_dict = torch.load(cfg.model_save_path)
  new_state_dict = OrderedDict()
  for k, v in state_dict.items():
    name = k[7:] # remove module
    new_state_dict[name] = v
  model.load_state_dict(new_state_dict)'''

model.eval()

enc = tokenizer(list(test['merged_text']),
                   padding=True,
                   truncation=True,
                   max_length=cfg.token_max,
                   return_tensors="pt")

test_ds = TensorDataset(enc['input_ids'], enc['attention_mask'])
test_dataloader = torch.utils.data.DataLoader(test_ds, batch_size=16, shuffle=False)


predictions = []
with torch.no_grad():
    for input_ids, attention_mask in test_dataloader:
        input_ids = input_ids
        attention_mask = attention_mask

        outputs = model(input_ids, attention_mask)
        predictions.extend(outputs.cpu().numpy())

RuntimeError: ignored